In [18]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [19]:
# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0

In [20]:
torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [21]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head, dropout=0.0):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # apply self attention and dropout
        x = x + self.dropout(self.sa(self.ln1(x)))
        # apply feedforward and dropout
        x = x + self.dropout(self.ffwd(self.ln2(x)))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self, dropout=0.0):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head, dropout) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx





In [22]:
import json
import os

dropout_values = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]

for dropout in dropout_values:
    print(f"Training model with dropout={dropout}")
    
    # 使用当前的dropout值实例化模型
    model = BigramLanguageModel(dropout=dropout).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
    
    # 用于存储当前dropout实验的损失值
    train_losses = []
    val_losses = []

    for iter in range(max_iters):
        # 每隔一定迭代次数评估训练和验证集上的损失
        if iter % eval_interval == 0 or iter == max_iters - 1:
            losses = estimate_loss()  # 确保这个函数里的model是当前循环的model
            train_losses.append(losses['train'].item())
            val_losses.append(losses['val'].item())
            print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
        
        # 采样一个数据批次
        xb, yb = get_batch('train')
        
        # 计算损失
        logits, loss = model(xb, yb)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()


    # 准备保存结果
    results = {"train_losses": train_losses, "val_losses": val_losses}

    # 定义文件名和路径
    folder_path = "./loss"
    filename = f'dropout_{dropout}.json'
    file_path = os.path.join(folder_path, filename)
    
    # 确保目录存在
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    
    # 将结果写入到文件中
    with open(file_path, 'w') as f:
        json.dump(results, f)
    
    print(f"Results for dropout={dropout} saved to {file_path}")



Training model with dropout=0.0
step 0: train loss 4.4116, val loss 4.4022
step 100: train loss 2.6568, val loss 2.6670
step 200: train loss 2.5091, val loss 2.5058
step 300: train loss 2.4194, val loss 2.4333
step 400: train loss 2.3498, val loss 2.3559
step 500: train loss 2.2965, val loss 2.3129
step 600: train loss 2.2411, val loss 2.2503
step 700: train loss 2.2054, val loss 2.2192
step 800: train loss 2.1639, val loss 2.1872
step 900: train loss 2.1247, val loss 2.1516
step 1000: train loss 2.1037, val loss 2.1303
step 1100: train loss 2.0700, val loss 2.1191
step 1200: train loss 2.0387, val loss 2.0802
step 1300: train loss 2.0246, val loss 2.0634
step 1400: train loss 1.9935, val loss 2.0368
step 1500: train loss 1.9711, val loss 2.0311
step 1600: train loss 1.9612, val loss 2.0467
step 1700: train loss 1.9404, val loss 2.0133
step 1800: train loss 1.9084, val loss 1.9934
step 1900: train loss 1.9091, val loss 1.9881
step 2000: train loss 1.8839, val loss 1.9943
step 2100: tra

In [24]:
import json
import matplotlib.pyplot as plt
import os

# 指定存放loss JSON文件的文件夹路径
folder_path = "./loss"
# 指定可视化结果保存的路径
visualization_save_path = "./visualization/loss_curves.png"

# 列出所有的JSON文件
loss_files = [f for f in os.listdir(folder_path) if f.endswith('.json')]

# 可视化每个dropout配置的损失曲线
plt.figure(figsize=(10, 6))

for file in loss_files:
    file_path = os.path.join(folder_path, file)
    
    # 读取JSON文件中的损失值
    with open(file_path, 'r') as f:
        losses = json.load(f)
    
    # 绘制训练和验证损失曲线
    # plt.plot(losses['train_losses'], label=f"Train Loss {file}")
    plt.plot(losses['val_losses'], label=f"Validation Loss {file}")

plt.title("Loss Curves for Different Dropout Configurations")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.grid(True)

# 保存图像
plt.savefig(visualization_save_path)
plt.close()

print(f"Visualization saved to {visualization_save_path}")


Visualization saved to ./visualization/loss_curves.png
